# Creating SPMD parallelism using OpenMP **parallel** and **teams** directive

From this part, we begin to introduce how to use OpenMP directives to write programs. We first introduce the most basic and most commonly used **parallel** directive and **teams** directive.

## Semantics and Syntax

### **parallel** Directive
The **parallel** directive is used to mark a parallel region. When a thread encounters a parallel region, a group of threads is created to execute the parallel region. Its syntax in C is:

```
#pragma omp parallel [clause[ [,] clause] ... ] new-line
    structured-block
```
The syntax in Fortran is:
```
!$omp parallel do [clause[ [,] clause] ... ]
    loop-nest
[!$omp end parallel do]
```
The original thread that executed the serial part will be the primary thread of the new team. All threads in the team execute parallel regions together. After a team is created, the number of threads in the team remains constant for the duration of that parallel region.
Within a parallel region, the thread number uniquely identifies each thread. A thread can obtain its own thread number by calling the *omp_get_thread_num* library routine.
> Primary thread is also known as the master thread

When a thread team is created, the primary thread will implicitly create as many tasks as the number of threads, each task is assigned and bounded to one thread.
When threads are all occupied, implicit tasks that have not been allocated will be suspended waiting for idle threads.

### **teams** Directive
The **teams** directive indicates that the loop that follows is split among multiple thread teams, one thread team computing one part of the task. Developers can use the **teams** directive to use a large number of thread teams.
Its syntax is:
```
#pragma omp teams [clause[ [,] clause] ... ] new-line
    structured-block
```
The syntax in Fortran is:
```
!$omp teams [clause[ [,] clause] ... ]
    loosely-structured-block
!$omp end teams
```
The following figure shows the execution model of the **teams** directive:
![teams_directive](teams.jpeg "topic1")

A league of teams is created when a thread encounters a **teams** construct. Each team is an initial team, and the initial thread in each team executes the team area.
After a team is created, the number of initial teams remains the same for the duration of the **teams** region.
Within a **teams** region, the initial team number uniquely identifies each initial team. A thread can obtain its own initial team number by calling the *omp_get_team_num* library routine.
The teams directive has the following characteristics:
- the **teams** directive can spawn one or more thread teams with the same number of threads
- code is portable for one thread team or multiple thread teams
- only the primary thread of each team continues to execute
- no synchronization between thread teams
- programmers don't need to think about how to decompose loops

OpenMP was originally designed for multithreading on shared-memory parallel computers, so the parallel directive only creates a single layer of parallelism.
The team instruction is used to express the second level of scalable parallelization, which can be used not only on the CPU, but also on the GPU.


## Clauses

## Examples